In [10]:
# Import dependencies

import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import json
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

ModuleNotFoundError: No module named 'gmaps'

In [7]:
# Read in the weather data / cities CSV that we created in Part 1

cities = pd.read_csv('Output_CSV/weather_data.csv', encoding="utf-8")
cities.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Kapaa,90,US,1592523413,74,22.08,-159.32,82.4,17.22
1,1,Portland,75,US,1592523303,31,45.52,-122.68,82.0,5.82
2,2,Saldanha,0,ZA,1592523566,45,-33.01,17.94,62.6,11.41
3,3,Mar del Plata,27,AR,1592523564,93,-38.00,-57.56,54.0,8.05
4,4,Nānākuli,20,US,1592523568,48,21.39,-158.15,87.8,10.29


In [8]:
# Create humidity heatmap

humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Lat", "Lng"]]

In [ ]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

In [ ]:
# Create a new dataframe that fits weather criteria
narrowed_city_df = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) & (cities["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

In [ ]:
# Store everything into variable named hotel_df, set parameters, and begin to plot markers on top of heatmap

hotellist = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]

In [ ]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig